# COGS 118B - Final Project

# Super Mario Bros With Reinforcement Learning

https://github.com/JaigonRyu/COGS188_group_template/tree/main

## Group members

- Jacob Hanshaw
- Jonathan Ito
- Hiroki Ito
- Rebecca Nguyen

# Abstract 

Our project focuses on developing a reinforcement learning (RL) agent to play Super Mario Bros using Stable-Retro. The objective is to optimize the agent’s ability to navigate levels efficiently, avoid obstacles, and maximize rewards. To make informed decisions, the agent receives pixel-based observations and game state data, including Mario’s position, enemy locations, and score.

We implemented a policy-based and value-based learning approach, Deep Q-Networks (DQN), and Double DQN (DDQN). Custom reward functions were designed to reinforce beneficial behaviors, and various hyperparameters were tested to enhance learning efficiency.

Performance was evaluated based on key metrics such as distance traveled, level completion efficiency, and total rewards accumulated. 

Our findings showed that DDQN and DQN had similar initial results. However, as we went through more iterations, we saw that the DDQN began to substantially outperform the DQN. We saw that our successful models often have a period where the results begin to decrease until they hit a spike where the performance dramatically increases. With increased training time, we were able to see that our models were being trained properly and learned to complete the levels


# Background

Artificial intelligence (AI) in gaming has been the subject of extensive research for years and its applications are expanding across various industries. While classic games like Super Mario Bros. may appear simple, they involve complex decision-making due to multiple inputs and dynamic elements such as obstacles, enemies, power-ups, lives, and timers. Demonstrating AI’s ability to handle these challenges effectively highlights its potential for navigating dynamic environments.

Among various AI techniques, reinforcement learning (RL) has emerged as one of the most efficient and widely explored methods. Previous research has applied RL to multiple games, including 
Pokémon <a name="Kalose"></a>[<sup>[2]</sup>](#Kalose), 
Super Mario Bros.<a name="Liao"></a>[<sup>[3]</sup>](#Liao), 
and Street Fighter<a name="Huertas"></a>[<sup>[1]</sup>](#Huertas), 
with Q-learning and other RL-based strategies playing a central role. Many studies indicate that Deep Reinforcement Learning with Double Q-Learning (DDQN) tends to be the most effective model for game-based learning tasks. However, we aimed to test how different RL models perform when compared with one another. This study<a name="De-Yu"></a>[<sup>[4]</sup>](#De-Yu) examines the differences in performance between Deep Q-Networks (DQN) and DDQN under various movement strategies, measuring how efficiency and learning outcomes differ between these approaches. By comparing our results with prior research, we aim to validate expected trends and analyze any variations in performance.

While we draw inspiration from existing studies, our approach involves designing a custom RL agent and conducting independent experiments. Specifically, we evaluate how our agent, trained with modified hyperparameters and reward structures, compares to results from prior research. This allows us to assess the effectiveness of our model in learning optimal gameplay strategies.

We chose Super Mario Bros. as our testing environment for several reasons. One key factor is its structured action space and consistency between episodes. Unlike games like Pokémon or Street Fighter, where AI agents must react to an unpredictable opponent (a "player 2" or CPU-controlled character), Super Mario Bros. offers a more controlled environment where the primary challenge is navigating the level efficiently. Training AI models for games with reactive opponents often require significantly more computational power and extended training time. Given our resource constraints, we focused on Super Mario Bros. as a more feasible environment for testing reinforcement learning models while still presenting meaningful challenges in sequential decision-making.





# Problem Statement

The problem that our group is attempting to solve is Super Mario Bros. Specifically, we are trying to train an agent (Mario) to be able to run through levels using reinforcement learning. Instead of focusing on a single level for the agent to train on, it will move through each level sequentially. This means that once it has completed level one (reached the flag), it will automatically be placed at the start of level two. This will go on until the episode ends (the agent reaches the maximum number of steps or it dies). Each game state is represented as an array with shape (240, 256, 3). The first two axes are for the shape of the window and the third axis is for color, as the game is in RGB color. This representation of the game allows us to pass each state into our models with ease.

As we will describe later, the agent will have access to a certain number of actions in order to complete its goal. These actions are represented as integers and don't necessarily have to be only one button press. For example, the movement "right and A" can be a singular action represented by some integer (such as 2). We can control which actions the agent has access to, meaning we are able to control the complexity of the action space.

The agent will receive rewards according to different rewards structures as well. The agent can obtain rewards by defeating enemies, progressing through the level, completing the level quickly, getting coins, etc. A combination of these metrics (defined by each rewards structure) is how we measure the success of the agent. Additionally, the gym environment that we use allows us to render each episode, allowing us to see exactly what the agent is doing. So the rewards are not only recorded, they can be seen happening as well.

# Data

We utilized the gym_super_mario_bros package as our dataset, which is built on nes-py, a framework that emulates the classic Super Mario Bros. game for reinforcement learning experiments.

## Data Preprocessing

### State Space Creation

For our data, we used the games' pixel frames to represent states. However, the original size of the frames was too large to compute efficiently. To fix this, we used gyms wrappers and a custom SkipFrame wrapper to preprocess after creating the environment. The following wrappers were applied:

**SkipFrame**:

SkipFrame allows the use of an action for one frame and its application to as many as we specify in the parameter. We decided to skip four frames, which saved us from redundancy. Rewards from the skipped states are merged, and we return the cumulative reward and the most recent step info. 

**ResizeObservation**:

This wrapper allows us to resize each frame from the original RGB input shape of (240, 256, 3) to the shape (84, 84, 3). This size reduction will help reduce computational costs later on with our neural networks. 

**GrayScaleObservation**:

Gray scaling our images does a similar thing by reducing the shape of our yet again. This time, the three RGB channels will be reduced to one grayscale channel, making our frame shape now (84, 84, 1).

**FrameStack**:

Frame stacking helps our network see "motion" better. To do this, we will stack four consecutive frames on each other, making our new shape (4, 84, 84, 1). It is important to note that our original skip frame means that each new frame state represents 16 actual game frames due to the effect of both skipping and stacking frames.


### Action Space Creation

 For our action space, `gym_super_mario_bros` comes with three discrete action spaces labeled RIGHT_ONLY, SIMPLE_MOVEMENT, and COMPLEX_MOVEMENT. To use them in our code, we had to use a `nes-py` wrapper called JoypadSpace. We initiated the wrapper with the RIGHT_ONLY and SIMPLE_MOVEMENT for our tests.

`RIGHT_ONLY = [ 
    ['NOOP'],
    ['right'],
 ['right', 'A'],
 ['right', 'B'],
 ['right', 'A', 'B'],
]`

`SIMPLE_MOVEMENT = [
    ['NOOP'],
    ['right'],
 ['right', 'A'],
 ['right', 'B'],
 ['right', 'A', 'B'],
    ['A'],
    ['left'],
]`

The code above shows that each action space has only 5-7 actions. Reducing our action space will make it easier for our agent to learn, given the more complex state space.

# Proposed Solution

### DQN Solution

A Deep Q-Network (DQN) is a clear potential solution to our problem. Due to the size of the game, Super Mario Bros has high dimensionality, which neural networks are equipped to handle. DQNs can estimate the Q-values for a state-action space over time, proving very effective for complex games. The neural network learns how to best approximate the Q-values so that it is not necessary to store all values directly (which would require far too much space).

### How the DQN Works

The neural network, or the policy network, used in the DQN will be built using PyTorch. Specifically, we will use Convolutional Neural Networks (CNNs) to extract the most important features from each frame. We input the frame dimensions into the CNN, which will then feed into a Linear model, which will have an output with the action space dimensions. The activation function we will use is ReLU, and we will be using the Adam optimizer.

Besides the neural network, there are a few key features of the DQN. First is the Q-function which represents the maximum cumulative reward that the agent can achieve from a state by taking a certain action. There is also a replay buffer that stores past experiences (state, action, reward, next state) that the model samples from during training. This is used to help optimize the model with gradient descent. Importantly, the DQN computes the Q-values using the same network that is being trained. More specifically, it selects an action that maximizes the Q-value and also estimates the future reward using the same policy network.

### DDQN Solution

We use a DDQN (Double Deep Q-Network) and compare it to the original DQN. A DDQN works similarly to a DQN except that it uses two different neural networks to select the best action and then evaluate the result of that action instead of one to do both tasks. This helps to reduce the overestimation of the Q-values which should stabilize learning. Our goal for the DDQN is the same as the DQN. We want to fill out our complex action value function with many state-action pairs. As already said, we still use neural networks because of their ability to handle high dimensionality.

### How the DDQN Works

The CNN is still the base of our DDQN solution. CNNs are great at extracting visual features from our frames. Our input to the CNN will be the frame dimensions, and the output will be the action space shape. The value in each of the output neurons represents a predicted q value for that action during that state.

We will use two networks to accomplish this:

- The first network is the online network. This network is the one that we will be training. The second is the target network, which contains the ground truth for the predictions in the loss function. 
- The target network will occasionally copy the weights of the online network to increase stability while training. To do this, the target network samples a random (state, action, reward, next_state, done) tuple from the replay buffer. This state in the tuple is passed to the online network to get the predicted q values for each action. However, we only focus on the action taken in our original tuple. To check its accuracy, we use the next state in the tuple to get a q value from the target network instead of checking all actions for the best q value. We then compute the target value using the function r + gamma * max_next_actionQ(next_state, next_action). This value is then passed into the loss function with the predicted Q(s, a) from earlier in the online network.

Note about loss and optimizer function: We used the Adam optimizer for our online network and MSE loss as our loss function.


# Evaluation Metrics

The main evaluation method we considered is the reward and the number of iterations required to get to a certain reward threshold. I.e even if 2 models reached a reward of 500, the one that got there quicker was more optimal. 
The reward function utilized in the initial DQN and DDQN models belongs to the Super Mario gym environment <a name="Kauten"></a>[<sup>[5]</sup>](#Kauten). This model looked at the change in x position, time remaining in the iteration, and if the agent dies.  
In subsequent tests, we created 3 custom reward functions, and tested using the DDQN to evaluate how they would react to different reward methods: 
- RewardMoveRight: Motivated to move right, punished heavily if it moves left. Uses a timer method to encourage speed. 
- RewardScoreBased: Awarded points depending on coin count, in-game score (awarded by coins, speed, power-ups, and killing enemies), and getting power-ups.
- RewardAntiDeath: Punished heavily upon death, received rewards depending on how many lives remain, reward depending on the time remaining.


# Results

### DQN

We first train the agent using a DQN to get a benchmark to compare with the DDQN. The hyperparameters we use for this baseline model are as follows: Batch Size = 512, Gamma = 0.99, Epsilon Start = 1.0, Epsilon End = 0.05, Epsilon Decay = 5000, Tau = 0.005, and the Learning Rate = 1e-4. We also use a memory size of 20000 and train the model over 2000 episodes. These hyperparameters ensure the agent explores a lot at the beginning of training but then settles into exploitation towards the end. Additionally, this model is trained using SIMPLE_MOVEMENT, which is described above.

#### Training Results

![Training Results](rewards_plot_dqn.png)

This is a plot of the rewards per episode obtained by the agent throughout the 2000 training episodes. While the reward is highly variable from episode to episode, we can see an obvious upward trend as the episode number increases. It starts out averaging at approximately 1,400 but ends averaging at around 2,500. Towards the end of training, the average reward drops slightly. It would have been interesting to see what happened in later episodes after this drop, but time constraints prevented us from exploring that. 

One thing that can be seen from the plot is the variability of the rewards. The agent will occasionally have a very good or bad episode. The peak reward that an agent obtained during an episode was just over 5,000, which is more than double what the average reward was at the end. Additionally, the worst reward that the agent received during an episode was around 300. Interestingly, this low reward was received near the end of training where we would expect the agent to be receiving high rewards. This variability is likely due to the epsilon-greedy selection method where the agent will, with a probability determined by an epsilon threshold, choose a random action. The particularly bad episodes may have occurred because of a string of these random actions that resulted in very bad states (from a reward maximization perspective).

Regardless of the variability of the rewards, this model offers a good baseline for us to compare the DDQN and any rewards function changes with. 

### DDQN

The first trial with the DDQN was a simple baseline with parameters of lr=0.00025, gamma=0.9, epsilon=1.0, eps_decay=0.99999975, eps_min=0.1, replay_buffer_capacity=100_000, batch_size=32, sync_network_rate=10000. We ran this configuration for 15,000 episodes where it routinely achieved a finish and had the following reward curve.

![Original DDQN](DDQN_plots\mario_env_15000_iters_rewards.png)

However, this curve was still spiky and did not have a significantly high average reward. To alter this trend, we ran using new hyperparameters. They were lr=0.0001, gamma = 0.95, epsilon = 1.0, epsilon_decay = 0.99999, eps_min = 0.1, replay_buffer_capacity=100_000, batch_size=64, sync_network_rate=5000. The slower learning rate will stabilize the training and the gamma increase will help value long-term rewards more which is important in Mario. We also lowered the epsilon decay speed-up convergence and encouraged less exploration. We increased the batch size to 64 to reduce variance in updates at the cost of power. The last parameter we changed is to lower the sync rate so the target network will update quickly. With these changes, the model performed better with a higher average score in less time. 

![New Hyperparameters DDQN](DDQN_plots\mario_env_newhyper_rewards.png)

The graph shows a better average score than all other previous models. This mainly can be attributed to the increase in batch size and the network sync rate. The batch size increase allowed for more stable updates and the sync rate also helped with this tremendously.  Along with the changes to make our model converge faster we discovered a good set of hyperparameters to work with for testing reward functions.

The last alteration we tried with DDQN was using a different reward function. To do this, we created a custom wrapper that would reward the agent based on score. However, after 2500 iterations it was clear that the reward function was leading us to poor performance and rewards as shown in the graph below.

![New Reward Functions](DDQN_plots\mario_env_score_rewards.png)

### Reward Functions

As mentioned earlier, we attempted 3 different reward functions with varying results. 

RewardMoveRight utilized an epsilon of 0.7, an epsilon min of 0.05, and an epsilon decay of 0.95 on SIMPLE_MOVEMENT. We hoped to reinforce its actions with riskier exploration. We also did not include a death penalty in hopes that the agent would continue to explore the environment quickly.  We ran this model for ~600 episodes. 

There were several issues that we ran into. When we initialized the model, we punished the agent for passing a certain time threshold. Since there was no death penalty, the agent would look to die when the time boundary would approach. After about 500 episodes, the model began to converge on that action, and the model plateaued. However, due to the limited episodes, it is difficult to conclude whether or not the model would have improved or continued to plateau. 

RewardScoreBased aimed to optimize speed, while still completing some amount of “human-like” gameplay. Instead of speedrunning, we wanted the model to collect coins, jump on enemies, and collect power-ups while still trying to finish the levels.

This model did not have a time, death, or position parameter. Due to this, there was no urgency for the model to move forward in the stage. After around 2500 iterations, we found that the model had converged and created a habit of collecting coins and killing enemies to a certain point, then dying.

RewardAntiDeath was designed to heavily punish death and did not take time into account. The goal was to encourage slow and safe behavior using a low epsilon and low learning rate. However, the code required access to certain gym instances that were blocked by a privacy error. As there was no way for us to access this data, we were unable to apply this.

We would like to note that as we only ran a limited number of iterations, these results are close to negligible. With more iterations, we believe that with fine-tuning of the functions, the models would be able to complete the level, however, it would be less efficient than utilizing the given reward function.


# Discussion

### Interpreting the result

Based on our models, it seems like the DDQN does better than the DQN when given the correct parameters. With the original hyperparameters that we tried for the DDQN, it performed worse than the DQN when measured on the average reward at the end of training. This is true even though the DDQN was trained on substantially more episodes than the DQN as well. The total reward for the DDQN also was even more variable than the DQN episode-by-episode reward. Throughout the entire training process the agent would have multiple episodes where it obtained near-zero rewards, even towards the end of the training process. This was a clear indication that we needed to change the hyperparameters.

After changing the hyperparameters of the DDQN, we saw it outperformed the DQN. The new stability in training, thanks to the batch size and sync rate changes, boosted the DDQN heavily. While the DDQN did seem to have a slower start than the DQN, it performed much better later. This is likely due to the way that the DQN overestimates Q-values while the DDQN does not (due to the two neural networks it uses). This overestimation makes the DQN look like it’s doing very well at the start, but then it plateaus as the overestimation bias builds up. On the other hand, the DDQN avoids overestimation so it starts slower. However, its training is more stable and accurate than the DQN’s, meaning that in later stages of training it outperforms the DQN because it hasn’t accrued overestimation bias. 

Looking at how the DDQN did with the new reward functions, we see some interesting observations. The RewardScoreBased function looked promising at first but ended up doing much worse than the DQN. As the number of episodes increased, the average reward it received only got lower. This likely means that the reward function was overweighting or underweighting some aspects. As said before, the reward function didn’t have a time, death, or position parameter. At least one of these is likely crucial for the successful learning of the model, which is why it performed so poorly. We would assume that the death parameter is the culprit, however, we would need more testing to know for sure. It is not fair to compare the DDQN using the RewardMoveRight function as it was only trained using 600 episodes. However, we would guess that it would also perform worse than the DQN because of how it started to plateau at only 600 episodes.

### Limitations

Our methods, though some worked and others looked very promising, were limited by the number of iterations we could run. The RL model required a substantial amount of computational power that we did not have. One example of this is the RewardMoveRight model which was trained for over 20 hours and only ended with ~600 episodes. This limited the number of hyperparameters we could test and how often we could fine-tune the models, as each test took an extensive time to complete. 

As Mario utilizes a continuous state space, we must break down the state into smaller chunks. By altering the replay buffer size as well as the batch size, we were able to better represent the state space and our models improved dramatically. If we were able to further increase this, our models would be better optimized and learn more efficiently. 

### Future work

As we continue to work on this model, we will try utilizing a virtual machine connected through UCSD’s network. This should allow us to run our model much quicker. We would also optimize the existing code to use less computation power, allowing us to test different hyperparameters to fine-tune our models. Fixing certain actions as a result of the reward functions would give insight into whether the model’s learning was plateauing due to convergence, or if it was a temporary setback and required more iterations. 

Ideally, all models should complete at least 5000 iterations, as in the documentation, many of the models we have read have taken somewhere in that range. We only had one model trained for 5000 iterations, and it was evident that that model significantly outperformed all other iterations we had.   

### Ethics & Privacy 

While developing our reinforcement learning (RL) agent for Super Mario Bros., we took several measures to ensure ethical considerations and privacy compliance in our approach.

Reinforcement learning (RL) agents optimize based on reward structures, such as greedy action, where short-term gains are prioritized. This could lead to behaviors where the RL agent exploits loopholes to maximize its reward in ways that deviate from the intended task. For instance, the agent might prioritize repeatedly collecting coins rather than completing levels efficiently. To mitigate this, we developed and tested multiple custom reward functions to guide the agent toward desirable behaviors, such as completing levels, over short-term gains, such as repetitive coin collection or intentional deaths. However, we observed instances where the agent exploited certain reward structures. Future work should refine these reward functions to further minimize unintended behaviors. We also implemented epsilon-greedy exploration to balance optimizing known rewards and exploring new behaviors outside the local optimum. We also analyzed the reward distribution across different training runs to ensure the agent learned generalized strategies rather than exploiting the reward function.

To ensure a fair evaluation of DQN and DQQN, we maintained a consistent action space across models, where applicable, to confirm that performance differences were due to learning improvements rather than an expanded action set. However, we adjusted hyperparameters in the DDQN model to improve stability and performance. Although training duration varied, we aimed to optimize each model independently to achieve the best possible results.  

Reproducibility is an obstacle in RL, as minor changes in hyperparameters can significantly alter performance. To promote transparency and ethical AI practices, we documented all hyperparameters and training methodologies to allow replication and verification of our results. Furthermore, we compared our results to prior research on RL in Super Mario Bros. to ensure consistency.

Training deep RL models can be computationally expensive and have environmental implications. To reduce resource waste, we attempted to optimize preprocessing steps to reduce computation load while retaining learning quality. We also balanced efficiency and training performance by carefully choosing batch size and replay memory.

Since we are using Stable-Retro to emulate Super Mario Bros, we adhered to all licensing agreements associated with Stable-Retro and gym_super_mario_bros. Our agent was trained on publicly available, simulated game environments, ensuring no proprietary or personally identifiable information (PII) was collected.

### Conclusion 

In this project, we explored and compared the performance of Deep Q-Networks (DQN) and Double Deep Q-Networks (DDQN) in training a reinforcement learning (RL) agent to play Super Mario Bros. using Stable-Retro. Our results demonstrated that while DQN performed better initially, DDQN ultimately outperformed it with improved stability and higher rewards after tuning the hyperparameters. Although DDQN has a slower start than DQN, it outperforms in later stages of training, suggesting that DDQN mitigates overestimation bias and enhances training efficiency in RL tasks, which aligns with our prior research. 

We also developed custom reward functions to optimize gameplay behavior, however, some reward structures led to unintended behaviors, such as collecting coins and killing enemies over level completion. Other reward structures looked promising initially but ultimately led to lower rewards over time, likely due to the lack of key parameters such as time constraints, death penalties, etc. This emphasizes the importance of carefully selecting reward function components to produce effective strategies.

Our research has given insight into reinforcement learning applications in gaming, more specifically in optimizing agent performance using different RL architectures and reward structures. Due to the computational and time limitations, future work should focus on running longer training sessions, refining reward functions, and exploring different hyperparameters to improve RL agent efficiency and decision-making.


# Footnotes

<a name="Huertas"></a>1.[^](#Huertas): Huertas, Zambrano, and Diaz Salamanca. Deep Reinforcement Learning for Optimal Gameplay in Street Fighter III: A Resource-Constrained Approach, repositorio.uniandes.edu.co/entities/publication/cb08b574-f75e-4d50-9c43-469099ec6795. Accessed 15 Feb. 2025.<br>

<a name="Kalose"></a>2.[^](#Kalose): Kalose, Akshay, et al. “Optimal Battle Strategy in Pokémon Using Reinforcement ” Optimal Battle Strategy in Pokemon Using Reinforcement Learning, web.stanford.edu/class/aa228/reports/2018/final151.pdf. Accessed 15 Feb. 2025. <br>

<a name="Liao"></a>3.[^](#Lias): Liao, Yizheng, et al. CS229 Final Report Reinforcement Learning to Play Mario, cs229.stanford.edu/proj2012/LiaoYiYang-RLtoPlayMario.pdf. Accessed 15 Feb. 2025. <br>

<a name="De-Yu"></a>4.[^](#De-Yu): Yu, De. Playing Super Mario Bros with Deep Reinforcement Learning, https://www.analyticsvidhya.com/blog/2021/06/playing-super-mario-bros-with-deep-reinforcement-learning

Accessed 15 Feb. 2025. <br>

<a name="Kauten"></a>5.[^](#Kauten) Kauten, C. Super Mario Bros for OpenAI Gym. GitHub. https://github.com/Kautenja/gym-super-mario-bros. Accessed 15 Feb. 2025. <br>

<a name="Sourish Kundu"></a>7.[^]Sourish Kundu Super-Mario-Bros-RL. GitHub. https://github.com/Sourish07/Super-Mario-Bros-RL. <br>

OpenAI. (2025). _ChatGPT [Large language model]. For plotting reward use.
